In [ ]:
import pandas as pd

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + 'bd71f17b286ef5148e2d7bf9fe27783f'}
    result = requests.get(url, headers=headers).json()
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

# Read addresses from Excel file into a DataFrame
file_path = "/content/destination.xlsx"  # Replace with your file path
df = pd.read_excel(file_path)

# Create empty lists to store latitude and longitude values
latitudes = []
longitudes = []

# Perform address to latitude and longitude conversion for each address in the DataFrame
for index, row in df.iterrows():
    address = row['address']  # Replace 'Address_column_name' with the actual column name in your Excel file
    try:
        lat, lon = addr_to_lat_lon(address)
        latitudes.append(lat)
        longitudes.append(lon)
    except IndexError:
        latitudes.append(None)
        longitudes.append(None)

# Add latitude and longitude columns to the DataFrame
df['Latitude'] = latitudes
df['Longitude'] = longitudes

# Save the DataFrame with latitude and longitude information to a new Excel file
output_file_path = "output_with_coordinates.xlsx"
df.to_excel(output_file_path, index=False)

print(f"Latitude and longitude values added to the DataFrame. Output saved to {output_file_path}")

Latitude and longitude values added to the DataFrame. Output saved to output_with_coordinates.xlsx


In [ ]:
import pandas as pd
import requests
import json

def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    # Replace {Issued Key} with your actual Kakao API key
    headers = {"Authorization": "KakaoAK {bd71f17b286ef5148e2d7bf9fe27783f}"}
    api_json = json.loads(requests.get(url, headers=headers).text)

    if 'documents' in api_json and api_json['documents']:
        address_info = api_json['documents'][0]['address']
        crd = {"lat": str(address_info['y']), "lng": str(address_info['x'])}  # latitude, longitude
        return crd
    else:
        return None

def process_excel_file(file_path):
    # Read Excel file into a DataFrame
    df = pd.read_excel(file_path)

    # Create new columns for latitude and longitude
    df['Latitude'] = None
    df['Longitude'] = None

    # Iterate through each row and update latitude and longitude
    for index, row in df.iterrows():
        address = row['address']  # Replace with the actual column name in your Excel file
        coordinates = get_location(address)

        if coordinates:
            df.at[index, 'Latitude'] = coordinates['lat']
            df.at[index, 'Longitude'] = coordinates['lng']

    # Save the updated DataFrame back to the Excel file
    df.to_excel('output_file.xlsx', index=False)

# Replace 'your_excel_file.xlsx' with the actual file path
process_excel_file('/content/destination.xlsx')